In [107]:
import pandas as pd
from ast import literal_eval
survey_df = pd.read_csv(r'C:\Users\01din\PycharmProjects\thesis\data\survey_results\survey_results.csv')
generic = lambda x: literal_eval(x)
conv = {'nutrition' : generic, 'steps' : generic, 'ingredients' : generic, 'id_column' : generic, 'jaccard_similarity' : generic, 'diff' : generic, 'recipes' : generic, 'ingredients_original' : generic, 'tags': generic}
df = pd.read_csv(r"C:\Users\01din\PycharmProjects\thesis\data\cleaned_recipes\recipes_with_JS.csv", converters=conv)
df.drop(['Unnamed: 0'], inplace=True, axis=1)
df_ingredients = pd.read_csv(r"C:\Users\01din\PycharmProjects\thesis\data\ingredients\ingredients_nutrition.csv")
df_ingredients.drop(['Unnamed: 0'], inplace=True, axis=1)
df_ingredients['nutrition'] = df_ingredients['nutrition'].apply(lambda x: literal_eval(x) if pd.notnull(x) else x)

Load all neccessary libraries and make sure everything runs on GPU

In [113]:
import torch
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [114]:
torch.cuda.is_available()

True

In [115]:
import torch
print(torch.__version__)

2.0.1+cu117


In [123]:
# Merge dfs
df_merged = df.merge(survey_df, on='id')

# Create input for model. It is essentially steps, replaced, and replacement ingredient, all seperated by seperator tokens
df_merged['input'] = df_merged['steps'].apply(' '.join) + ' [STEPS] ' + df_merged['replaced'] + ' [REPLACED] ' + df_merged['replacement'] + ' [REPLACEMENT] '

# Convert Y/N to binary labels for the classifier
df_merged['result'] = df_merged['result'].map({'Yes': 1, 'No': 0})

# Load bert tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Split the dataset into train/test
train_text, test_text, train_labels, test_labels = train_test_split(df_merged['input'], df_merged['result'], test_size=0.2)

class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(self.texts[idx], return_tensors='pt', padding='max_length', truncation=True, max_length=512)
        return {'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
                'labels': torch.tensor(self.labels[idx])}

# Create data loaders
batch_size = 8
train_dataset = TextClassificationDataset(train_text.tolist(), train_labels.tolist(), tokenizer)
test_dataset = TextClassificationDataset(test_text.tolist(), test_labels.tolist(), tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

# Load bert model and convert to GPU
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model = model.to(device)
# Init optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

This was not used

In [122]:
epochs = 3
# Train the model
model.train()
torch.cuda.empty_cache()
for epoch in range(epochs):
    total_loss = 0
    progress_bar = tqdm(train_dataloader, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})




model.eval()
predicted_labels = []
true_labels = []

with torch.no_grad():
    progress_bar = tqdm(test_dataloader, desc='Testing', leave=False, disable=False)
    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)

        predicted_labels.extend(preds.tolist())
        true_labels.extend(labels.tolist())

# Convert lists to np
predicted_labels = np.array(predicted_labels)
true_labels = np.array(true_labels)

# Calculate accuracy
accuracy = np.sum(predicted_labels == true_labels) / len(true_labels)
print(f'Accuracy: {accuracy}')

# Calculate per-class accuracy, precision, recall, and F1-score
class_names = ['No', 'Yes']  # Replace with your actual class names
classification_rep = classification_report(true_labels, predicted_labels, target_names=class_names)
print('Classification Report:')
print(classification_rep)

# Calculate the confusion matrix
confusion_mat = confusion_matrix(true_labels, predicted_labels)
print('Confusion Matrix:')
print(confusion_mat)


KeyboardInterrupt: 

In [124]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
import copy
epochs = 100
# This allows to track loss over multiple epochs
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

# Set patience for early stopping and init vars
patience = 5
best_valid_loss = float('inf')
best_model = None
counter = 0

for epoch in range(epochs):
    total_loss = 0
    progress_bar = tqdm(train_dataloader, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

    # After each epoch, check the validation loss and see how it compares
    valid_loss = 0
    model.eval()
    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            valid_loss += loss.item()

    # Average validation loss
    avg_valid_loss = valid_loss / len(test_dataloader)
    print(f'Validation Loss: {avg_valid_loss}')

    # Update learning rate
    scheduler.step(avg_valid_loss)

    # Check for early stopping
    if avg_valid_loss < best_valid_loss:
        best_valid_loss = avg_valid_loss
        counter = 0  # Reset counter if validation loss improved
        best_model = copy.deepcopy(model)
    else:
        counter += 1  # Increment counter if validation loss did not improve
        print(f'EarlyStopping counter: {counter} out of {patience}')
        if counter >= patience:
            print('Early stopping')
            break  # Break loop if counter has reached the limit of patience

    model.train()  # Turn on training mode for next epoch

# After stopping, save best model
model = best_model

Validation Loss: 0.6844920378464919


Validation Loss: 0.697747698196998
EarlyStopping counter: 1 out of 5


Validation Loss: 0.6998969454031724
EarlyStopping counter: 2 out of 5


KeyboardInterrupt: 

In [21]:
model.save_pretrained(r"C:\Users\01din\PycharmProjects\thesis\models\classifier")

Loading trained model

In [73]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained(r"C:\Users\01din\PycharmProjects\thesis\models\classifier")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

Method which tests all possible replacements (all ingredients from the same category except ones with the same USDA ingredient) and classifies them as 1/0.
Return with confidence score

In [54]:
def test_replacements(df, recipe_id, df_ingredients, ingredient_to_replace, model, tokenizer, device):
    recipe_row = df[df['id'] == recipe_id]

    original_recipe = recipe_row['steps'].apply(' '.join).iloc[0]
    original_ingredient_desc = df_ingredients[df_ingredients.ingredient == ingredient_to_replace].FdGrp_Desc.iloc[0]
    original_ingredient_long_desc = df_ingredients[df_ingredients.ingredient == ingredient_to_replace].Long_Desc.iloc[0]
    all_predictions = []

    for i, row in tqdm(df_ingredients.iterrows(), total=df_ingredients.shape[0]):
        if row['FdGrp_Desc'] == original_ingredient_desc and row['Long_Desc'] != original_ingredient_long_desc:
            replacement_ingredient = row['ingredient']
            if replacement_ingredient != ingredient_to_replace:
                # Format the recipe text
                test_recipe = original_recipe + ' [REPLACED] ' + ingredient_to_replace + ' [REPLACEMENT] ' + replacement_ingredient
                encoding = tokenizer.encode_plus(test_recipe, return_tensors='pt', padding='max_length', truncation=True, max_length=512)
                input_ids = encoding['input_ids'].to(device)
                attention_mask = encoding['attention_mask'].to(device)

                model.eval()
                with torch.no_grad():
                    outputs = model(input_ids, attention_mask=attention_mask)
                    logits = outputs.logits
                    _, predicted_labels = torch.max(logits, dim=1)
                    confidence = torch.nn.functional.softmax(logits, dim=1)[:, 1].item()  # The confidence of label 1

                all_predictions.append({
                    'replacement_ingredient': replacement_ingredient,
                    'predicted_label': predicted_labels.item(),
                    'confidence': confidence
                })

    # Sort by confidence for label 1
    all_predictions.sort(key=lambda x: -x['confidence'] if x['predicted_label'] == 1 else 1)

    return all_predictions


In [119]:
results = test_replacements(df, 263659, df_ingredients, 'cashews', model, tokenizer, device)
results

100%|██████████| 2717/2717 [00:08<00:00, 327.90it/s]


These ids, and replaced, were found randomly sampled.

In [81]:
ids = [263659, 344624, 19881, 470964, 168586, 34403, 448396, 253207, 256098, 209474]
replaced = ['cashews', 'diced tomatoes', 'red bell pepper', 'ground beef', 'worcestershire sauce', 'sugar', 'dijon mustard', 'unsalted butter', 'vegetable broth', 'yellow onion']

# Df for output
df_output = pd.DataFrame(columns=['id', 'replaced', 'replacements'])

# Loop over all ids
for i in range(len(ids)):
    results = test_replacements(df, ids[i], df_ingredients, replaced[i], model, tokenizer, device)

    # Extract first 5 dictionaries and then 'replacement_ingredient' key from each
    replacements = [result['replacement_ingredient'] for result in results[:10]]
    df_output = df_output.append({'id': ids[i], 'replaced': replaced[i], 'replacements': replacements}, ignore_index=True)

100%|██████████| 2717/2717 [00:02<00:00, 1053.36it/s]
C:\Users\01din\AppData\Local\Temp\ipykernel_23844\2417914801.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_output = df_output.append({'id': ids[i], 'replaced': replaced[i], 'replacements': replacements}, ignore_index=True)
100%|██████████| 2717/2717 [00:20<00:00, 131.89it/s]
C:\Users\01din\AppData\Local\Temp\ipykernel_23844\2417914801.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_output = df_output.append({'id': ids[i], 'replaced': replaced[i], 'replacements': replacements}, ignore_index=True)
100%|██████████| 2717/2717 [00:20<00:00, 134.60it/s]
C:\Users\01din\AppData\Local\Temp\ipykernel_23844\2417914801.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inst

In [82]:
df_output

,id,replaced,replacements
0,263659,cashews,"[pecan pieces, pecans, unsalted cashews, roast..."
1,344624,diced tomatoes,"[cherry tomatoes, diced tomato, mexican-style ..."
2,19881,red bell pepper,"[pimientos, jalapeno chiles, jalapeno, pimento..."
3,470964,ground beef,"[hamburger meat, extra lean ground beef, lean ..."
4,168586,worcestershire sauce,"[sriracha sauce, heinz 57 steak sauce, adobo s..."
5,34403,sugar,"[unbleached cane sugar, demerara sugar, turbin..."
6,448396,dijon mustard,"[dry mustard, dried mustard, ground mustard, s..."
7,253207,unsalted butter,"[half-and-half, fontina, medium cheddar, gruye..."
8,256098,vegetable broth,"[canned chicken broth, fat free chicken broth,..."
9,209474,yellow onion,"[sweet potato, chives, new potato, fresh chive..."


This part was never used in evaluations, but this was a way to output many recipes and all of its classifications

In [ ]:
df_sample = df.sample(2000)

In [42]:
import json
import random

json_data_list = []


for index, row in df_sample.iterrows():
    # Get data necessary
    name = row['name']
    steps = row['steps']
    ingredients_original = row['ingredients_original']

    # exclude spices & herbs
    non_spice_ingredients = [ingredient for ingredient in ingredients_original if df_ingredients[df_ingredients['ingredient'] == ingredient]['FdGrp_Desc'].iloc[0] != 'Spices and Herbs']

    # Get random replacement string.
    if non_spice_ingredients:  # Checks if there are non spice ingredients
        replaced = random.choice(non_spice_ingredients)
    else:
        continue  # Skip if no non spice ingredients

    # Get replacements
    id = row['id']  # Get the id from the row.
    replacements = test_replacements(df, id, df_ingredients, replaced, model, tokenizer, device)

    # Format for json
    data = {
        'id': id,
        'name': name,
        'steps': steps,
        'ingredients_original': ingredients_original,
        'replaced': replaced,
        'replacements': replacements
    }

    # Convert to json and add to list
    json_data = json.dumps(data)
    json_data_list.append(json_data)

100%|██████████| 2717/2717 [00:08<00:00, 302.53it/s]

{"id": 235653, "name": "sinfully rich   easy sticky buns", "steps": ["thaw bread dough", "grease 9\" x 13\" pan", "tear one loaf of bread up into pieces and put in bottom of pan", "melt butter in milk in small sauce-pan on stove top on medium heat", "add brown sugar & cinnamon to melted butter & milk", "add pudding and stir until dissolved", "pour over torn-up bread dough in pan", "tear up 2nd loaf of bread and place on top", "cover with wax paper and let rise until double", "bake at 350 degrees for 30 minutes", "remove from oven and loosen edges of pan with butter knife", "immediately turn upside down onto a cookie sheet covered with wax paper"], "ingredients_original": ["frozen bread dough", "butter", "vanilla pudding mix", "milk", "cinnamon", "light brown sugar"], "replaced": "light brown sugar", "replacements": [{"replacement_ingredient": "light-brown sugar", "predicted_label": 1, "confidence": 0.955348014831543}, {"replacement_ingredient": "palm sugar", "predicted_label": 1, "conf

In [43]:
with open(r'C:\Users\01din\PycharmProjects\thesis\data\results_to_evaluate\classifier_results.json', 'w') as json_file:
    json.dump(json_data_list, json_file)